In [1]:
!pip install -qU \
  langchain-core \
  langchain-google-genai \
  langchain-community \
  scikit-image


In [2]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

Enter API key for Google Gemini: ········


In [3]:
prompt = """

Answer the user's query based on the context below.
If you don't know the answer based on the context
provided, please respond, I don't know.

---
Context: {context}
---
"""

In [4]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
        ("system", prompt),
        ("user", "{query}")
])

In [5]:
prompt_template.input_variables

['context', 'query']

In [6]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="\n\nAnswer the user's query based on the context below.\nIf you don't know the answer based on the context\nprovided, please respond, I don't know.\n\n---\nContext: {context}\n---\n"), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

In [7]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)

prompt_template_2 = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(prompt),
        HumanMessagePromptTemplate.from_template("{query}")
])

In [8]:
prompt_template_2.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="\n\nAnswer the user's query based on the context below.\nIf you don't know the answer based on the context\nprovided, please respond, I don't know.\n\n---\nContext: {context}\n---\n"), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

In [9]:
prompt_template_2.input_variables

['context', 'query']

In [10]:
from langchain.chat_models import init_chat_model


llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai", temperature=0.0) 

In [11]:
pipeline = (
    {
        "query": lambda x: x["query"],
        "context": lambda x: x["context"]
    }
    | prompt_template_2 
    | llm)

In [12]:
context = """
Aurelio AI is an AI company developing tooling for AI
engineers. There focus is on the language AI with the team having strong
expertise in building AI agents and a strong background in information
retreival.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI 
Platform providing engineers with tooling to help them build with 
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI become langchain expert in September 2024 after a long 
track record of delivering AI solutions built with Langchain ecosystem.

"""

query ="What does Aurelio AI do?"

In [13]:
pipeline.invoke({"query": query, "context": context})

AIMessage(content='Aurelio AI is an AI company that develops tooling for AI engineers, with a focus on language AI. They build AI agents and have a strong background in information retrieval. They are the creators of Semantic Router and Semantic Chunkers, and offer an AI Platform with tools for building with AI. Additionally, they provide development services to help other organizations bring their AI tech to market.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--345b4504-8067-4b2a-babb-8c4ccc657057-0', usage_metadata={'input_tokens': 194, 'output_tokens': 76, 'total_tokens': 270, 'input_token_details': {'cache_read': 0}})

In [14]:
pipeline.invoke({"query": "What does Microsoft do", "context": context})

AIMessage(content="I don't know. The context provided describes Aurelio AI, not Microsoft.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--6bdbbf55-67a1-4b87-8dda-9aa8bd3823f3-0', usage_metadata={'input_tokens': 191, 'output_tokens': 18, 'total_tokens': 209, 'input_token_details': {'cache_read': 0}})

## Few Short Prompting

In [15]:
example_prompt = ChatPromptTemplate.from_messages([ 
        HumanMessagePromptTemplate.from_template("{input}"),
        AIMessagePromptTemplate.from_template("{output}")
])

In [16]:
examples = [
    {"input": "Here is the query #1", "output": "Here is the answer #1"},
    {"input": "Here is the query #2", "output": "Here is the answer #2"},
    {"input": "Here is the query #3", "output": "Here is the answer #3"}    
]

In [17]:
from langchain.prompts import FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples
)

print(few_shot_prompt.format())

Human: Here is the query #1
AI: Here is the answer #1
Human: Here is the query #2
AI: Here is the answer #2
Human: Here is the query #3
AI: Here is the answer #3


In [18]:
new_system_prompt = """

Answer the user's query based on the context below.
If you don't know the answer based on the context
provided, please respond, I don't know.

Always answser in markdown format. When doing so please
provide headers, short summaries, follow with bullet 
points, then conclude.

---
Context: {context}
---
"""

prompt_template_markdown = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(new_system_prompt),
        HumanMessagePromptTemplate.from_template("{query}")
])
 

pipeline_markdown = (
    {
        "query": lambda x: x["query"],
        "context": lambda x: x["context"]
    }
    | prompt_template_markdown 
    | llm)

out = pipeline_markdown.invoke({"query": query, "context": context}).content

print(out)

# Aurelio AI Overview

Aurelio AI is an AI company specializing in tooling for AI engineers, particularly in the realm of language AI.

## Summary

*   Focus on language AI with expertise in AI agents and information retrieval.
*   Develops open-source frameworks like Semantic Router and Semantic Chunkers.
*   Offers an AI Platform with tools for AI development.
*   Provides AI development services to other organizations.
*   Became a Langchain expert in September 2024.

## Core Activities

*   **Open Source Frameworks:** Creating and maintaining tools like Semantic Router and Semantic Chunkers.
*   **AI Platform:** Providing a platform with various tools to help engineers build AI solutions.
*   **Development Services:** Assisting other organizations in bringing their AI technology to market.

## Conclusion

Aurelio AI empowers AI engineers through open-source tools, a comprehensive AI platform, and expert development services, with a strong focus on language AI and a proven track rec

In [19]:
from IPython.display import display, Markdown

display(Markdown(out))

# Aurelio AI Overview

Aurelio AI is an AI company specializing in tooling for AI engineers, particularly in the realm of language AI.

## Summary

*   Focus on language AI with expertise in AI agents and information retrieval.
*   Develops open-source frameworks like Semantic Router and Semantic Chunkers.
*   Offers an AI Platform with tools for AI development.
*   Provides AI development services to other organizations.
*   Became a Langchain expert in September 2024.

## Core Activities

*   **Open Source Frameworks:** Creating and maintaining tools like Semantic Router and Semantic Chunkers.
*   **AI Platform:** Providing a platform with various tools to help engineers build AI solutions.
*   **Development Services:** Assisting other organizations in bringing their AI technology to market.

## Conclusion

Aurelio AI empowers AI engineers through open-source tools, a comprehensive AI platform, and expert development services, with a strong focus on language AI and a proven track record in the Langchain ecosystem.

In [20]:
examples = [
    {
        "input": "Can you explain gravity?",
        "output": (
            "## Gravity\n\n"
            "Gravity is one of the fundamental forces in the universe.\n\n"
            "### Discovery\n\n"
            "* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n"
            "* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n"
            "### In General Relativity\n\n"
            "* Gravity is described as the curvature of spacetime.\n"
            "* The more massive an object is, the more it curves spacetime.\n"
            "* This curvature is what causes objects to fall towards each other.\n\n"
            "### Gravitons\n\n"
            "* Gravitons are hypothetical particles that mediate the force of gravity.\n"
            "* They have not yet been detected.\n\n"
            "**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n"
        )
    },
    {
        "input": "What is the capital of France?",
        "output": (
            "## France\n\n"
            "The capital of France is Paris.\n\n"
            "### Origins\n\n"
            "* The name Paris comes from the Latin word \"Parisini\" which referred to a Celtic people living in the area.\n"
            "* The Romans named the city Lutetia, which means \"the place where the river turns\".\n"
            "* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.\n\n"
            "**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.\n\n"
        )
    }
]

In [21]:
few_shot_prompt_new = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples
)

out = few_shot_prompt_new.format()

display(Markdown(out))

Human: Can you explain gravity?
AI: ## Gravity

Gravity is one of the fundamental forces in the universe.

### Discovery

* Gravity was first discovered by Sir Isaac Newton in the late 17th century.
* It was said that Newton theorized about gravity after seeing an apple fall from a tree.

### In General Relativity

* Gravity is described as the curvature of spacetime.
* The more massive an object is, the more it curves spacetime.
* This curvature is what causes objects to fall towards each other.

### Gravitons

* Gravitons are hypothetical particles that mediate the force of gravity.
* They have not yet been detected.

**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.


Human: What is the capital of France?
AI: ## France

The capital of France is Paris.

### Origins

* The name Paris comes from the Latin word "Parisini" which referred to a Celtic people living in the area.
* The Romans named the city Lutetia, which means "the place where the river turns".
* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.

**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.



In [28]:
few_shot_prompt_template = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(new_system_prompt),
        FewShotChatMessagePromptTemplate(
            example_prompt = example_prompt,
            examples = examples
        ),
        HumanMessagePromptTemplate.from_template("{query}")
])

In [29]:
few_shot_prompt_template

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="\n\nAnswer the user's query based on the context below.\nIf you don't know the answer based on the context\nprovided, please respond, I don't know.\n\nAlways answser in markdown format. When doing so please\nprovide headers, short summaries, follow with bullet \npoints, then conclude.\n\n---\nContext: {context}\n---\n"), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': 'Can you explain gravity?', 'output': '## Gravity\n\nGravity is one of the fundamental forces in the universe.\n\n### Discovery\n\n* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n### In General Relativity\n\n* Gravity is described as the curvatur

In [30]:
pipeline_few_shot = (
    {
        "query": lambda x: x["query"],
        "context": lambda x: x["context"]
    }
    | few_shot_prompt_template 
    | llm)

out = pipeline_few_shot.invoke({"query": query, "context": context}).content

display(Markdown(out))

## Aurelio AI

Aurelio AI is an AI company focused on providing tools and services for AI engineers, particularly in the realm of language AI.

### Summary

*   **Focus:** Language AI, AI Agents, and Information Retrieval.
*   **Open Source Frameworks:** Developers of Semantic Router and Semantic Chunkers.
*   **AI Platform:** Offers tooling for AI engineers to build with AI.
*   **Development Services:** Provides AI development services to other organizations.
*   **Langchain Expertise:** Became Langchain experts in September 2024.

**In conclusion,** Aurelio AI empowers AI engineers through open-source tools, a dedicated AI platform, and expert development services, with a strong focus on language AI and the Langchain ecosystem.